In [133]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [134]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [135]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [136]:
df.shape

(50000, 2)

In [137]:
# One review
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

## I. Text Cleaning

1. Selecting Sample of 10000 rows out of 50000 rows.
2. Remove html tags
3. Remove special characters
4. Converting every thing to lower case
5. Removing Stop words
6. Stemming

In [138]:
df=df.sample(10000)

In [139]:
df.shape

(10000, 2)

In [140]:
df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [141]:
df.head()

,review,sentiment
7428,They make everything too easy in this film. It...,0
48141,"I must admit, I was one of the skeptics who pr...",1
15686,This is the type of late-night cable flick usu...,0
26936,Will they ever make movies without nudity and ...,0
12907,"What can I say that hasn't been said about ""Th...",1


In [142]:
# Example to clean html tags

import re
clean = re.compile('<.*?>')
re.sub(clean, '', df.iloc[2].review)

"This is the type of late-night cable flick usually associated with Andrew Stevens or Shannon Tweed. Though unlike most of Tweed and Stevens' T&A fueled vehicles, this is lethally dull!Let's cut to the chase. The real reason for watching this non-thriller is to see Shannon Doherty's breasts. Anyone who states otherwise is a LIAR! However, most of her steamy sex scenes appear to be all smoke and mirrors.Notice that all the shots where her head and chest show at the same time are quick peek-a-boo flashes. The frames where the camera lingers on her nude body, there's little or no face attached or she's behind a dripping, wet shower door.All you boob-watchers out there know what that means - Body double!I must admit though, that the finale where Doherty is bound, blindfolded and menaced with a knife, provided a certain fetishistic thrill.If you find a VHS copy anywhere, buy it! As all involved are probably too embarrassed to ever let this come out on DVD!"

In [143]:
## Function to clean html tags

def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [144]:
# cleaning html tags from 'review' column

df['review']=df['review'].apply(clean_html)

In [145]:
## converting everything to lower

def convert_lower(text):
    return text.lower()

In [146]:
df['review']=df['review'].apply(convert_lower)

In [147]:
## function to remove special characters

def remove_special(text):
    x=''
    
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x + ' '
    return x

In [148]:
# Example

remove_special(' th%e @ classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare. forget pretty pictures painted for mainstream audiences, f')

' th e   classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare  forget pretty pictures painted for mainstream audiences  f'

In [149]:
df['review']=df['review'].apply(remove_special)

In [150]:
## Remove the stop words

import nltk
from nltk.corpus import stopwords
def remove_stopwords(text):
    x=[]
    for i in text.split():
        
        if i not in stopwords.words('english'):
            x.append(i)
    y=x[:]
    x.clear()
    return y

In [151]:
df['review']=df['review'].apply(remove_stopwords)

In [152]:
df.head()

,review,sentiment
7428,"[make, everything, easy, film, sort, reminded,...",0
48141,"[must, admit, one, skeptics, prematurely, judg...",1
15686,"[type, late, night, cable, flick, usually, ass...",0
26936,"[ever, make, movies, without, nudity, sex, cam...",0
12907,"[say, said, haunting, everything, would, make,...",1


In [153]:
## Perform stemming

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [154]:
y=[]
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z

In [155]:
stem_words(['I','loved','loving','it'])

['I', 'love', 'love', 'it']

In [156]:
df['review']=df['review'].apply(stem_words)

In [157]:
## Join back: Converting Back to string from list

def join_back(list_input):
    return " ".join(list_input)

In [158]:
df['review']=df['review'].apply(join_back)

In [159]:
df['review']

7428     make everyth easi film sort remind indiana jon...
48141    must admit one skeptic prematur judg show rel ...
15686    type late night cabl flick usual associ andrew...
26936    ever make movi without nuditi sex came 3 00 su...
12907    say said haunt everyth would make great pictur...
                               ...                        
8426     eric stoltz deliv extraordinari perform joel g...
31771    care like comment think write movi good obsess...
48800    three stoog featur length western comedi music...
2118     basic movi might one mesmer titl made either t...
46512    movi terribl brain surfin dude inherit compani...
Name: review, Length: 10000, dtype: object

In [160]:
X=df.iloc[:,0:1].values

In [161]:
X.shape

(10000, 1)

## II. Vectorizing using Bag of Words.

In [163]:
from sklearn.feature_extraction.text import CountVectorizer
# Taking 2500 best words
cv=CountVectorizer(max_features=2500)

In [164]:
X=cv.fit_transform(df['review']).toarray()

In [165]:
X.shape

(10000, 2500)

In [167]:
y=df.iloc[:,-1].values

In [168]:
y.shape

(10000,)

## III. Applying Naive Bayes

In [170]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [171]:
X_train.shape

(8000, 2500)

In [172]:
X_test.shape

(2000, 2500)

In [173]:
y_train.shape

(8000,)

In [174]:
y_test.shape

(2000,)

In [177]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [178]:
clf1=GaussianNB()
clf2=MultinomialNB()
clf3=BernoulliNB()

In [179]:
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)

BernoulliNB()

In [180]:
y_pred1=clf1.predict(X_test)
y_pred2=clf2.predict(X_test)
y_pred3=clf3.predict(X_test)

In [181]:
y_test.shape

(2000,)

In [182]:
y_pred1.shape

(2000,)

In [183]:
from sklearn.metrics import accuracy_score

In [185]:
print("Gaussian:",accuracy_score(y_test,y_pred1))
print("Multinomial:",accuracy_score(y_test,y_pred2))
print("Bernaulli:",accuracy_score(y_test,y_pred3))

Gaussian: 0.733
Multinomial: 0.83
Bernaulli: 0.8335
